In [1]:
from mpi4py import MPI
from phe import paillier

import numpy as np


In [5]:
data_1 = np.random.uniform(0, 2, size = (1000, 2))
data_2 = np.random.uniform(0,2,size=(1000,2))
target = np.random.randint(0, 2, size=(1000,1))

combined_data = np.concatenate((data_1, target), axis=1)

num_rows, num_cols = data_1.shape

In [10]:
# finding the split for the current threshold
def split_indexing(X, feature_index, threshold, indices):
    binary_masking = [0 if X[i, feature_index] <= threshold else 1 for i in indices]
    return binary_masking

In [60]:
def sigmoid(x):
    return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

def gradient_calc(y, y_val, index):
    return y - y_val*index

def hess_calc(y, y_val, index):
    return -y/(y_val*index)**2


In [61]:
def finding_current_values(y, binary_masking):
    # split the target according to the binary mask
    full_vec = np.ones_like(binary_masking)
    right_vals = y*binary_masking
    left_vals = y - right_vals
    full_amount = y.T@full_vec
    right_amount = y.T@binary_masking
    left_amount = full_amount - right_amount
    full_num = len(full_vec)
    right_num = sum(binary_masking)
    left_num = full_num - right_num
    left_val = sigmoid(left_amount/left_num)
    right_val = sigmoid(right_amount/right_num)
    left_gradients = gradient_calc(y, left_val, left_vals)
    right_gradients = gradient_calc(y, right_val, right_vals)
    return  left_gradients, right_gradients


In [62]:
binarys = split_indexing(data_1, feature_index=0, threshold=1, indices=range(1000))
left_grads, right_grads = finding_current_values(target, np.array(binarys).reshape(-1,1))

In [72]:
target

array([[1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
    

In [71]:
target*np.array(binarys).reshape(-1,1)

array([[0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
    

In [66]:
left_grads

array([[0.36864765],
       [0.36864765],
       [1.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [1.        ],
       [1.        ],
       [0.36864765],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [0.36864765],
       [1.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.36864765],
       [0.        ],
       [0.36864765],
       [0.36864765],
       [1.        ],
       [0.36864765],
       [0.36864765],
       [0.36864765],
       [0.36864765],
       [1.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [1.        ],
       [0.36864765],
       [1.        ],
       [0.        ],
       [1.        ],
       [0.36864765],
       [0.        ],
       [0.        ],
       [0.368

In [ ]:
gradients = gradient_calc(y, pred_val, index)

use a lookup table to track the nodes and flow of the tree.
The look up will be looking up the tree id

tree id will give us a mapping of nodes, the nodes will map to party id and the party will have rhe feature and threshold value

{tree index : 

{node : (party_id, feature id, threshold)} 
    
}



In [13]:

# what do we need to do?
# create the encryption keys
# send the encrypted gradients and the encrypted hessian
# perform the split, all of the data will need to be allocated to the left or the right
# then need to query 
# only need to store the condition for split in the node
# the nodes contain the id of the data points that are in the node.
# so need to track where we end up.

# we have the tree, then we have the node, then we have the leaves

# a leaf is a node that does not have any next nodes.
# what is the valuation of each node?
# node valuation is known upon construct of the node


class Node():

    def __init__(self, left_node=None, right_node=None, feature=None, condition=None, thresh=None, value=None):
        self.feature = feature
        self.condition = condition
        self.threshold = thresh
        self.value = value

    def value_assignment(self, value):
        if self.left_Node == None and self.right_Node == None:
            self.value = value

    def extend_new_Node(self, leftNode, rightNode):
        self.left_Node = leftNode
        self.right_Node = rightNode

    def assign_split(self, condition, threshold, feature):
        self.feature = feature
        self.condition = condition
        self.threshold=threshold


class Tree():
    def __init__(self, root, next_tree=None):
        self.root = root
        self.next_tree=next_tree


In [14]:
current_val = np.round(sum(target)/len(target))
root = Node(value=current_val)

First_Tree = Tree(root=root)


In [33]:
def entropy(y_true, y_pred):
    return y_true(np.log(y_pred)) + (1-y_true)*np.log(1 - y_pred)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def gradient_entropy(y_true, y_pred):
    return y_true/y_pred - (1-y_true)/(1-y_pred)

def hessian(y_true, y_pred):
    return -y_true/(y_pred)**2 + (1-y_true)/(1 - y_pred)**2

In [19]:
# sketch assume uniform distribution for thresholding
thresholds = {}
for i in range(num_cols):
    thresholds[i] = np.linspace(min(data[:, i]), max(data[:,i]), 32)

If we assume the sketch holds would it imply that we can utilize the sketch for the entire tree building? 
The sketch and initial thresholding will give us probabilities of the thresholding so we gather the total data and can sketch as it would mean we assume similar distribution for the feature under all conditions, in the sense that the marginal for each feature.

We can simply put the data into the bins of the thresholding to decide the best split.

What do I actually need to track? How to efficiently keep the indices

The valuation of the individual nodes will the be average value of the node, so will be given by 

sum of all data points at the node / total number of data points at the node.

Then for future trees it will be a weighted sum of the valuations, and then will pass into the sigmoid function

So can calculate the valuation on the fly

The indexing of the datapoint is what is difficult, how to track the index.


For batches we will track index and left or right, or for each level we can track which node the data index will be in, 
so this will get communicated with reach pass through, so for node 1, you will receive a bunch of indices and will keep the ones that are valid.


Need to build a tree first, then the tree that is built will have residuals, misclassified data instances.
These will then be used to train the next tree.

Then passing all points through, the residuals of the results of both trees sill be used to train next one etc.

What if all the trees are initialized, so using hyper-parameters number of trees and depth, max things in node

We can initialize all of the trees and fill it in accordingly.

party 1 needs to find the indexing with respect to the thing, for each threshold would need to communicate the Lefts and rights, and then the thing would need to be performed

The gradients would need to be calculated for every threshold as the values will be different.
Because the thresholding will lead to different partitioning of the data instances so the predicted value for the lhs and rhs are different for each threshold for each feature, so gradient and hessian would need to be re-calculated.

In [ ]:
class GBDT():
    def __init__(self, num_trees = 5, max_depth = 5, min_points_node = 5):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.min_points = min_points_node




    def find_split(self, X):

        return optimal
    

class Node():
    def __init__(self, value, feature, threshold, left_N = None, right_N=None):
        self.value = value
        self.feature = feature
        self.thresh = threshold
        self.left_N = left_N
        self.right_N = right_N
    
    

In [22]:
First_Tree.root.value

loss_valuation = {}
best_split = {}

loss_value = target - target.mean()

for i in range(num_cols):
    left_valuation = {}
    right_valuation = {}
    for index, value in enumerate(data[:, i]):
        for id, threshold in enumerate(thresholds[i]):
            if value <= threshold:
                if id not in left_valuation:
                    left_valuation[id] = loss_value[index]
                else:
                    left_valuation[id] += loss_value[index]
            else:
                if id not in right_valuation:
                    right_valuation[id] = loss_value[index]
                else:
                    right_valuation[id] += loss_value[index]

    max_gains = None
    max_threshold_id = None
    for id in left_valuation:
        if max_gains == None:
            max_gains = left_valuation[id] + right_valuation[id]
            max_threshold_id = id
        else:
            if id not in left_valuation and id not in right_valuation:
                continue
            elif id not in left_valuation:
                current_gains = right_valuation[id]
            elif id not in right_valuation:
                current_gains = left_valuation[id]
            else:
                current_gains = left_valuation[id] + right_valuation[id]


            if current_gains > max_gains:
                max_gains = current_gains
                max_threshold_id = id

    loss_valuation[(i, id)] = max_gains

best_split = max(loss_valuation, loss_valuation.get)

print(best_split)

TypeError: '>' not supported between instances of 'builtin_function_or_method' and 'dict'

In [28]:
loss_valuation

{(0, 0): array([-27.801]), (1, 0): array([865.472])}